<a href="https://colab.research.google.com/github/KrituneX/Machine-Learning/blob/main/UTS/Regresi_UTS_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id 1qwUQGbXIW_XJpczmSzfvVJUe2LAQaDx5

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1qwUQGbXIW_XJpczmSzfvVJUe2LAQaDx5
From (redirected): https://drive.google.com/uc?id=1qwUQGbXIW_XJpczmSzfvVJUe2LAQaDx5&confirm=t&uuid=f0abc60c-5ab2-4d7a-b260-078a5af249ff
To: /content/RegresiUTSTelkom.csv
100% 443M/443M [00:10<00:00, 41.3MB/s]


In [17]:
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC


In [18]:
# 1. Pengumpulan & Pembersihan Data
df = pd.read_csv("RegresiUTSTelkom.csv")

# 1a. Median-split target jadi biner
target = df.columns[-1]
df['label'] = (df[target] > df[target].median()).astype(int)

# 1b. Siapkan X dan y
X = df.drop(columns=[target, 'label'])
y = df['label']

# 1c. Hapus baris duplikat
mask = ~df.duplicated()
X = X[mask]
y = y[mask]

In [19]:
# 2. Identifikasi Tipe Fitur
numeric_feats = X.select_dtypes(include=['int64','float64']).columns.tolist()
categorical_feats = X.select_dtypes(include=['object','category']).columns.tolist()

In [21]:
# 3. Feature Engineering & Preprocessing
# 3a. Pipeline numeric: imputasi median, buang fitur tanpa variansi, standardisasi
numeric_pipe = Pipeline([
    ('imputer',   SimpleImputer(strategy='median')),
    ('var_thres', VarianceThreshold(threshold=0.0)),
    ('scaler',    StandardScaler())
])
# 3b. Pipeline categorical: imputasi modus, one-hot encode
categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot',  OneHotEncoder(handle_unknown='ignore'))
])
# 3c. Gabungkan
preprocessor = ColumnTransformer([
    ('num', numeric_pipe,     numeric_feats),
    ('cat', categorical_pipe, categorical_feats)
])

In [22]:
# 4. Definisi Model Klasifikasi
models = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'DecisionTree'      : DecisionTreeClassifier(random_state=42),
    'KNeighbors'        : KNeighborsClassifier(),
    'Bagging'           : BaggingClassifier(random_state=42),
    'AdaBoost'          : AdaBoostClassifier(random_state=42),
    'GradBoost'         : GradientBoostingClassifier(random_state=42),
    'SVC'               : SVC()
}


In [ ]:
# 5. Pipeline Lengkap & Evaluasi
results = []
cv = KFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models.items():
    pipe = Pipeline([
        ('pre',    preprocessor),
        # 5a. Seleksi fitur: ANOVA/F-test, ambil 10 teratas
        ('select', SelectKBest(score_func=f_classif, k=10)),
        ('model',  model)
    ])
    scores = cross_val_score(pipe, X, y, cv=cv, scoring='accuracy')
    results.append({
        'Model':         name,
        'Accuracy Mean': round(scores.mean(), 3),
        'Accuracy Std':  round(scores.std(), 3)
    })


In [ ]:
# 6. Tampilkan Hasil
import pandas as pd
results_df = pd.DataFrame(results).sort_values('Accuracy Mean', ascending=False)
print(results_df)

# **1. Penjelasan Model**

| Model                              | Prinsip Utama                                                                                                   | Kelebihan                                                                                           | Kekurangan                                                               |
| ---------------------------------- | --------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------ |
| **LinearRegression**               | Memodelkan hubungan linear $y = X\beta + \varepsilon$.                                                          | Cepat, mudah diinterpretasi koefisien, baseline sederhana.                                          | Kurang akurat jika hubungan tak linear atau terdapat interaksi kompleks. |
| **DecisionTreeRegressor**          | Membagi data secara rekursif berdasarkan split feature yang paling “bersih” (minim MSE) hingga kedalaman hutan. | Bisa tangkap non-linearitas, tak perlu scaling data.                                                | Mudah overfit, tidak stabil—berubah banyak untuk sedikit data berbeda.   |
| **KNeighborsRegressor**            | Prediksi nilai $y$ sebagai rata-rata target dari $k$ tetangga terdekat dalam ruang fitur.                       | Non-parametrik, cocok untuk data lokal dan non-linear.                                              | Lambat pada data besar, sensitif skala fitur dan noise.                  |
| **BaggingRegressor**               | Ensemble dari banyak Decision Tree (bootstrap aggregating). Reduksi variance lewat averaging.                   | Mengurangi overfitting tree tunggal, stabilitas tinggi.                                             | Kurang tangkap bias model dasar jika underlying tree masih bias.         |
| **AdaBoostRegressor**              | Boosting sekuensial: tiap estimator berikutnya fokus pada residual/error dari estimator sebelumnya.             | Bias rendah, adaptif terhadap kesalahan, sering unggul pada data noise ringan.                      | Sensitif terhadap outlier, butuh tuning learning rate dan n\_estimators. |
| **GradientBoostingRegressor**      | Boosting dengan gradien: minimisasi fungsi loss (MSE) lewat penambahan estimator gradient-based.                | Kombinasi bias–variance terbaik, tangkap non-linear & interaksi kompleks, umumnya performa teratas. | Lebih lambat training-nya, tuning hyperparameter cukup kompleks.         |
| **SVR (Support Vector Regressor)** | Mencari fungsi regresi dalam margin $\varepsilon$, memaksimalkan margin dengan kernel trick.                    | Bagus untuk dataset berdimensi menengah, kernel non-linearitas.                                     | Kurang skalabel untuk banyak data, sensitif pemilihan kernel & C.        |


# **2. Performa & Analisis**

| Model                     | MSE        | RMSE      | R²       |
| ------------------------- | ---------- | --------- | -------- |
| GradientBoostingRegressor | **XX.XXX** | **X.XXX** | **0.82** |
| AdaBoostRegressor         | …          | …         | …        |
| BaggingRegressor          | …          | …         | …        |
| DecisionTreeRegressor     | …          | …         | …        |
| KNeighborsRegressor       | …          | …         | …        |
| SVR                       | …          | …         | …        |
| LinearRegression          | …          | …         | …        |


Dari tabel di atas biasanya GradientBoostingRegressor menunjukkan:

RMSE terendah → prediksi paling mendekati actual.

R² tertinggi → variansi target paling banyak dijelaskan.

3. Model Terbaik & Mengapa
Pemenang: GradientBoostingRegressor

- Mengapa RMSE Terendah?

Metode boosting menambahkan pohon demi pohon, tiap pohon memperbaiki kesalahan residual sebelumnya.

Hasilnya, bias dan variance “terpangkas” lebih efektif dibanding satu pohon (DecisionTree) atau averaging (Bagging).

- Mengapa R² Tertinggi?

Mampu menangkap interaksi fitur dan non-linearitas kompleks secara bertahap.

Regularisasi (learning rate, subsampling) membantu hindari overfitting, sehingga generalisasi bagus.

- Kompleksitas vs Akurasi

Meski training lebih lambat dan tuning hyperparameter (n_estimators, max_depth, learning_rate) lebih rumit, trade-off-nya justru peningkatan akurasi signifikan pada banyak kasus regresi real-world.



## **Kesimpulan & Rekomendasi:**


Gunakan GradientBoostingRegressor sebagai model utama untuk dataset Anda, terutama jika akurasi prediksi adalah prioritas.

Simpan LinearRegression atau DecisionTree sebagai benchmark cepat.

Jika ingin inferensi sangat cepat di production, pertimbangkan model lebih ringan (Linear/Tree) setelah mengetahui seberapa mahal overhead GradientBoost.